In [1]:
pip install unidecode scikit-learn BeautifulSoup4 pandas requests lxml selenium matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.preprocessing import StandardScaler
from requests.auth import HTTPBasicAuth
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
from io import StringIO
import pandas as pd
import numpy as np
import unidecode
import requests
import locale
import pickle
import re 
import lxml
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

In [17]:
import scrapy
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

class MySpider(scrapy.Spider):
    name = 'myspider'
    start_urls = ['https://www.senscritique.com/films/tops/top111']

    def __init__(self, *args, **kwargs):
        super(MySpider, self).__init__(*args, **kwargs)
        chrome_options = Options()
        chrome_options.add_argument('--headless')  # Run Chrome in headless mode (no GUI)
        self.driver = webdriver.Chrome(chrome_options=chrome_options)

    def parse(self, response):
        self.driver.get(response.url)

        # Scroll down to load all content (you might need to adjust the range or use a more sophisticated scroll strategy)
        for _ in range(8):
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            self.driver.implicitly_wait(2)  # wait for content to load

        # Get the fully loaded page source
        page_source = self.driver.page_source

        # Now you can use Scrapy selectors to extract data
        movie_titles = response.css('a.elpo-item-link::text').getall()
        yield {
            'movie_titles': movie_titles,
        }

    def closed(self, reason):
        self.driver.quit()


In [ ]:
from selenium import webdriver

driver = webdriver.Chrome()
driver.get('https://www.google.com')
driver.quit()


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome()

try:
    # Load the page
    driver.get('https://www.senscritique.com/films/tops/top111')

    # Scroll down to load all content (you might need to adjust the range or use a more sophisticated scroll strategy)
    for _ in range(8):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # wait for content to load

    # Now the page should be fully loaded
    page_source = driver.page_source

finally:
    # Always close the driver to clean up
    driver.quit()

# Now you can use BeautifulSoup to parse the HTML
soup = BeautifulSoup(page_source, 'html.parser')

print(soup)


<html lang="fr"><head><script id="impactify-sdk" src="https://ad.impactify.io/static/ad/tag.js"></script><script async="" src="https://static.smilewanted.com/js/decode_consent/decode_consent.js" type="text/javascript"></script><script async="" src="https://csync.smilewanted.com"></script><script async="" src="https://sync.richaudience.com/b3adde1f4bbb31c3485562d6e3ddceb4/js/?r=1698600831445"></script><script src="https://loadus.exelator.com/load/?p=204&amp;g=119&amp;buid=df-7dadb1e4-8f17-4bdc-819a-a917c04a9a54" type="text/javascript"></script><ins id="viralize-fallback-o1g1rm3vnd8"><script type="application/javascript">if(typeof top.pbShowHeroesCorner === "function")top.pbShowHeroesCorner();</script></ins><script data-wid="auto" src="https://ads.viralize.tv/display/?zid=AAE4fTfBToxH-tSS&amp;schain=1.0,1!cultureg.eu,243,1,,," style="display: none;" type="text/javascript"></script><script type="application/javascript">function pbImpactifyFooter(){ cgCallCorner('[object Object]')}</script

In [ ]:

def film_info(link):

    # combine them to the new url
    base_url = 'https://www.senscritique.com'
    film_url = base_url + link

    # convert the url content to soup object
    film_page = requests.get(film_url)
    film_page_soup = BeautifulSoup(film_page.content, "html.parser")

    # use a new dict to save info
    extrainfo_dict = {}

    extrainfo_dict['film_year'] = film_page_soup.find('p', {'class': 'Text__SCTitle-sc-1aoldkr-1 CoverProductInfos__StyledText-sc-1un0kh1-13 eGhlHy jugtWW'}).text
    
    original_title_element = film_page_soup.find('p', {'class': 'Text__SCTitle-sc-1aoldkr-1 CoverProductInfos__StyledText-sc-1un0kh1-13 eGhlHy kuMSsq'})
    extrainfo_dict['original_title'] = original_title_element.text if original_title_element else None

    stats_elements = film_page_soup.select('p.Text__SCText-sc-1aoldkr-0.Stats__Text-sc-1u6v943-2.gATBvI')
    if len(stats_elements) == 3:
        extrainfo_dict['stars'] = stats_elements[0].text
        extrainfo_dict['saves'] = stats_elements[1].text
        extrainfo_dict['favorites'] = stats_elements[2].text
    
    critique_number_text = film_page_soup.select('.NavigationTab__WrapperTextStyled-sc-18dtd9d-7')[2].text
    extrainfo_dict['critique_number'] = int(critique_number_text.split('(')[-1].split(')')[0])

    return extrainfo_dict

In [ ]:


count = 0


all_films_dict = {}
films = soup.find_all('div', class_="ProductListItem__Wrapper-sc-1jkxxpj-1 kusRkg")

for film in films:
        title = find_text(film, {'name': 'a', 'attrs': {"data-testid": "product-title"}})
        new_film_dict = {key: find_text(film, criteria) for key, criteria in search_criteria_dict.items()}

        link = film.find('a', {"data-testid": "product-title"})['href']

        new_film_dict.update(film_info(link))
        
        all_films_dict[title] = new_film_dict


print(all_films_dict)


In [ ]:
print(len(all_films_dict.keys()))

In [ ]:
import json
with open('film_info.json', 'w') as json_file:
    json.dump(all_films_dict, json_file, indent=4)


In [ ]:
df = pd.read_json('film_info.json', orient='index')

df.head()

In [ ]:
def convert_to_minutes(duration_str):
    parts = duration_str.split()
    hours = int(parts[0])
    minutes = int(parts[-2])
    total_minutes = hours * 60 + minutes
    return total_minutes

df['duration_minutes'] = df['duration'].apply(convert_to_minutes)


In [ ]:
df.head(5)

In [ ]:
df['ranking_number'] = df['ranking'].str.split('.').str.get(0)
#instead of using df['ranking'].str.split()[0]

df.head(5)

In [ ]:
df['genres_split'] = df['genres'].str.split(', ')
df_genre = df.explode('genres_split')

df_genre.head(5)

In [ ]:
def deal_with_k(number_str):
    if 'K' in number_str:
        parts = number_str.split('K')[0]
        return float(parts) * 1000
    else:
        return int(number_str)

df['stars_number'] = df['stars'].apply(deal_with_k)
df['saves_number'] = df['saves'].apply(deal_with_k)
df['favorites_number'] = df['favorites'].apply(deal_with_k)

df.head(5)


In [ ]:
df = df.drop(columns=['stars_number'])
